In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
def merge_files(filenames, output_filename):
    with open(output_filename, "w", encoding="utf-8") as outfile:
        for filename in filenames:
            with open(filename, "r", encoding="utf-8") as infile:
                outfile.write(infile.read())

filenames = ["f.csv","f1.csv","f2.csv","f3.csv","f4.csv","f5.csv","f6.csv","f7.csv","f8.csv","f9.csv",
             "f10.csv","f11.csv","f12.csv","f3.csv","f14.csv"]
output_filename = "merged_file.csv"
merge_files(filenames, output_filename)


In [3]:
df=pd.read_csv('merged_file.csv')
df.head(5)

,Product_Name,Selling Price,MRP,Items Bought Last Month,Ratings,Numeric_Ratings,Total Ratings,Brand,Offer%
0,1.5 Ton 3 Star AI Flexicool Inverter Split AC ...,33990.0,67790.0,600,"4.0 out of 5 stars 2,085",4.0,2085,Whirlpool,49.86
1,"1.5 Ton 3 Star Inverter Split AC (Copper, PM 2...",36990.0,58400.0,600,"4.0 out of 5 stars 1,454",4.0,1454,Voltas,36.66
2,1.5 Ton 3 Star Inverter Split AC (5 in 1 Conve...,32990.0,58990.0,500,"4.2 out of 5 stars 5,561",4.2,5561,LG,44.08
3,1.5 Ton 5 Star Wi-Fi Inverter Smart Split AC (...,42990.0,63400.0,500,"4.2 out of 5 stars 4,881",4.2,4881,Whirlpool,32.19
4,1.5 Ton 5 Star AI Flexicool Inverter Split AC ...,40990.0,76090.0,300,"4.0 out of 5 stars 1,531",4.0,1531,Haier,46.13


In [4]:
df.shape

(81915, 9)

In [5]:
df.isnull().sum()

Product_Name               0
Selling Price              0
MRP                        0
Items Bought Last Month    0
Ratings                    0
Numeric_Ratings            0
Total Ratings              0
Brand                      8
Offer%                     0
dtype: int64

In [6]:
df=df.dropna()

In [7]:
df.isnull().sum()

Product_Name               0
Selling Price              0
MRP                        0
Items Bought Last Month    0
Ratings                    0
Numeric_Ratings            0
Total Ratings              0
Brand                      0
Offer%                     0
dtype: int64

In [8]:
df.shape

(81907, 9)

# Label Encoding

In [9]:
# Encode brand names
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['Brand_Encoded'] = encoder.fit_transform(df['Brand'])

In [10]:
df.dtypes

Product_Name               object
Selling Price              object
MRP                        object
Items Bought Last Month    object
Ratings                    object
Numeric_Ratings            object
Total Ratings              object
Brand                      object
Offer%                     object
Brand_Encoded               int32
dtype: object

# Changing DataTypes

In [11]:
# Convert columns to the required data types
df['Selling Price'] = pd.to_numeric(df['Selling Price'], errors='coerce')
df['MRP'] = pd.to_numeric(df['MRP'], errors='coerce')
df['Items Bought Last Month'] = pd.to_numeric(df['Items Bought Last Month'], errors='coerce')
df['Total Ratings'] = pd.to_numeric(df['Total Ratings'], errors='coerce')
df['Offer%'] = pd.to_numeric(df['Offer%'], errors='coerce')
df['Numeric_Ratings'] = pd.to_numeric(df['Numeric_Ratings'], errors='coerce')
df['Brand'] = df['Brand'].astype(str)

In [14]:
df.dtypes

Product_Name                object
Selling Price              float64
MRP                        float64
Items Bought Last Month    float64
Ratings                     object
Numeric_Ratings            float64
Total Ratings              float64
Brand                       object
Offer%                     float64
Brand_Encoded                int32
dtype: object

In [15]:
df.isnull().sum()

Product_Name                0
Selling Price              14
MRP                        14
Items Bought Last Month    14
Ratings                     0
Numeric_Ratings            14
Total Ratings              14
Brand                       0
Offer%                     14
Brand_Encoded               0
dtype: int64

In [16]:
df=df.dropna()

In [17]:
df.isnull().sum()

Product_Name               0
Selling Price              0
MRP                        0
Items Bought Last Month    0
Ratings                    0
Numeric_Ratings            0
Total Ratings              0
Brand                      0
Offer%                     0
Brand_Encoded              0
dtype: int64

# Top 40 Brands

In [12]:
df['Brand'].value_counts()[:40]

Brand
Skechers         4253
Adidas           4211
Under Armour     4193
Reebok           4145
New Balance      4142
Nike             4139
Puma             4077
Vans             4074
HP               2353
Samsung          2211
Lenovo           1978
Acer             1650
Huawei           1407
Sony             1403
LG               1144
iPhone           1116
MSI              1100
Dell              994
Oppo              964
Panasonic         964
Motorola          940
ASUS              920
Vivo              920
Xiaomi            894
OnePlus           860
Realme            816
DELL              760
Philips           697
Skullcandy        695
RHA               689
Apple AirPods     678
Shure             677
Beats by Dre      664
Sennheiser        658
Bose              657
Jabra             639
Toshiba           592
ViewSonic         548
BenQ              547
AOC               534
Name: count, dtype: int64

In [13]:
df['Brand']=df['Brand'].replace('Dell','DELL')

# NLP

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [19]:
# Ensure NLTK resources are downloaded
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bittu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bittu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bittu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## NLP PreProcessing

In [20]:
# Define preprocess_text function
def preprocess_text(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters and punctuation
    tokens = word_tokenize(text)  # Tokenize the text
    stop_words = set(stopwords.words('english'))   # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()  # Lemmatize the tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    processed_text = ' '.join(tokens)  # Join tokens back into a string
    return processed_text


In [21]:
# Preprocess product names
df['Processed_Product_Name'] = df['Product_Name'].apply(preprocess_text)

In [22]:
# Vectorize the preprocessed product names
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Processed_Product_Name'])

# Recommendation System

In [23]:
# Function to recommend products based on brand and ratings
def recommend_products_(brand_name, min_rating=2.0, top_n=10000):
    filtered_df = df[(df['Brand'] == brand_name) & (df['Numeric_Ratings'] >= min_rating)]
    if filtered_df.empty:
        return "No products found for the given brand and rating criteria."
    else:
        recommended_products = filtered_df.sort_values(by='Numeric_Ratings', ascending=False).head(top_n)[['Product_Name', 'Numeric_Ratings']]
        return recommended_products

In [24]:
# User interaction
print("Welcome to the Product Recommender!")
print("Please enter the brand name:")
brand_name = input("Brand: ")
print("Please enter the minimum rating (e.g., 4.0):")
min_rating = float(input("Minimum Rating: "))


Welcome to the Product Recommender!
Please enter the brand name:


Brand:  Puma


Please enter the minimum rating (e.g., 4.0):


Minimum Rating:  2.0


# Recommended Products

In [25]:
# Recommend products
recommended_products = recommend_products_(brand_name, min_rating)
if isinstance(recommended_products, str):
    print(recommended_products)
else:
    print("\nRecommended Products:")
    print(recommended_products.to_string(index=False))

# Display all recommended products
print("\nAll Recommended Products:")
print(recommended_products)


Recommended Products:
                                                                                                                                                                                                                                   Product_Name  Numeric_Ratings
                                                                                                                                                                                              Womens Solid Slip-on Mules with Metal Accent Mule              5.0
                                                                                                                                                                              Mens Adispree 5.0 M TRAROY/MYSBLU/SILVMT Sneaker - 12 UK (CK9529)              5.0
                                                                                                                                                             Premium Men's Comfortable Clogs Sandals with Adju

### Final Dataset

In [26]:
df

,Product_Name,Selling Price,MRP,Items Bought Last Month,Ratings,Numeric_Ratings,Total Ratings,Brand,Offer%,Brand_Encoded,Processed_Product_Name
0,1.5 Ton 3 Star AI Flexicool Inverter Split AC ...,33990.00,67790.0,600.0,"4.0 out of 5 stars 2,085",4.0,2085.0,Whirlpool,49.86,3247,15 ton 3 star ai flexicool inverter split ac c...
1,"1.5 Ton 3 Star Inverter Split AC (Copper, PM 2...",36990.00,58400.0,600.0,"4.0 out of 5 stars 1,454",4.0,1454.0,Voltas,36.66,3189,15 ton 3 star inverter split ac copper pm 25 f...
2,1.5 Ton 3 Star Inverter Split AC (5 in 1 Conve...,32990.00,58990.0,500.0,"4.2 out of 5 stars 5,561",4.2,5561.0,LG,44.08,1730,15 ton 3 star inverter split ac 5 1 convertibl...
3,1.5 Ton 5 Star Wi-Fi Inverter Smart Split AC (...,42990.00,63400.0,500.0,"4.2 out of 5 stars 4,881",4.2,4881.0,Whirlpool,32.19,3247,15 ton 5 star wifi inverter smart split ac cop...
4,1.5 Ton 5 Star AI Flexicool Inverter Split AC ...,40990.00,76090.0,300.0,"4.0 out of 5 stars 1,531",4.0,1531.0,Haier,46.13,1344,15 ton 5 star ai flexicool inverter split ac c...
...,...,...,...,...,...,...,...,...,...,...,...
81910,Women's Deerskin Gore Front Moccasin Black Dee...,17564.26,0.0,100.0,4.1 out of 5 stars 9,4.1,9.0,Adidas,0.00,254,woman deerskin gore front moccasin black deers...
81911,Women s Deerskin Gore Front Moccasin Black Dee...,11234.50,0.0,100.0,4.3 out of 5 stars 20,4.3,20.0,Reebok,0.00,2494,woman deerskin gore front moccasin black deers...
81912,Women's Nurse Mates ANNIE LO Moccasins WHITE 8...,16290.00,0.0,100.0,5.0 out of 5 stars 6,5.0,6.0,New Balance,0.00,2069,woman nurse mate annie lo moccasin white 85
81913,Women's Chillax Washed Laceless Slip-On Sneake...,399.00,999.0,100.0,3.9 out of 5 stars 8,3.9,8.0,New Balance,60.06,2069,woman chillax washed laceless slipon sneakerdr...


### Saved File

In [27]:
df.to_csv('merged_file.csv',index=False)